In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import losses
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import tensorflow as tf
import generator

In [2]:
def generate_dataset():
    gen = generator.Generator(path_s="data/initial kernels/Kernel_Carbon_Adsorption.npy",
                        path_d="data/initial kernels/Kernel_Carbon_Desorption.npy",
                        path_p_d="data/initial kernels/Pressure_Carbon.npy",
                        path_p_s="data/initial kernels/Pressure_Carbon.npy",
                        path_a="data/initial kernels/Size_Kernel_Carbon_Adsorption.npy"
                )
    gen.generate_data_set()

def show_dataset():
        dataset = hkl.load('data/datasets/carbon3.hkl')
        i = 8
        plt.plot(dataset[i]["isotherm"], marker=".")
        plt.plot(dataset[i]["pore_distribution"], marker=".")
        plt.show()
        print(len(dataset))

In [3]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plt.legend()
    plt.grid(True)
    plt.yscale("log")
    plt.show()

In [4]:
min_exp_pressure_i = 48
max_exp_pressure_i = 546
with open('data/datasets/carbon3_right.npz', 'rb') as f:
        dataset = np.load(f)
        isotherm_data = dataset["isotherm_data"]
        pore_distribution_data = dataset["pore_distribution_data"]
x = np.empty((isotherm_data.shape[0], (-min_exp_pressure_i + max_exp_pressure_i)))
y = np.empty(pore_distribution_data.shape)
for i in range(len(isotherm_data)):
    isotherm = isotherm_data[i] #/ max(isotherm_data[i])
    pore_distribution = pore_distribution_data[i] #/ max(pore_distribution_data[i])
    isotherm = isotherm[min_exp_pressure_i:max_exp_pressure_i]
    x[i] = isotherm
    y[i] = pore_distribution
x, y = shuffle(x, y)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [5]:
norm_layer = layers.Normalization()
norm_layer.adapt(x)

In [45]:
### DENSE NET
model = keras.Sequential(
    [
        keras.Input(shape=len(x_train[0], )),
        norm_layer,
        layers.Dense(400, activation='relu'),
        layers.Dense(300, activation='relu'),
        layers.Dense(300, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(len(y_train[0])) #  activation=tf.math.abs
    ]
)
### CONV NET 
# x_train=np.array(x_train)
# x_test=np.array(x_test)
# x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
# x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
# def make_model(input_shape, norm_layer):
#     input_layer = keras.layers.Input(input_shape)
# 
#     conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
#     conv1 = keras.layers.BatchNormalization()(conv1)
#     conv1 = keras.layers.ReLU()(conv1)
# 
#     conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
#     conv2 = keras.layers.BatchNormalization()(conv2)
#     conv2 = keras.layers.ReLU()(conv2)
# 
#     conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
#     conv3 = keras.layers.BatchNormalization()(conv3)
#     conv3 = keras.layers.ReLU()(conv3)
# 
#     gap = keras.layers.GlobalAveragePooling1D()(conv3)
# 
#     output_layer = keras.layers.Dense(len(y_train[0]))(gap)
# 
#     return keras.models.Model(inputs=input_layer, outputs=output_layer)
# 
# norm_layer = keras.layers.Normalization(axis=None)
# norm_layer.adapt(np.array(x))
# model = make_model(input_shape=np.array(x_train).shape[1:], norm_layer=norm_layer)

In [46]:
model.compile(loss='mean_squared_error', optimizer='Adam')

In [48]:
mcp_save = keras.callbacks.ModelCheckpoint(filepath='data/models/carbon2_best.keras', save_best_only=True,
                                           monitor='val_loss', mode='min', verbose=0, save_weights_only=False,
                                           save_freq='epoch')
reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.85,
                                                   patience=8, verbose=1, mode='auto')
history = model.fit(np.array(x_train), np.array(y_train),
                    epochs=100, batch_size=128, shuffle=True,
                    validation_data=(np.array(x_test), np.array(y_test)),
                    callbacks=[mcp_save, reduce_lr_loss])

Epoch 1/100
586/586 [==============================] - 4s 6ms/step - loss: 0.0142 - val_loss: 0.0131 - lr: 0.0010
Epoch 2/100
586/586 [==============================] - 3s 5ms/step - loss: 0.0140 - val_loss: 0.0131 - lr: 0.0010
Epoch 3/100
586/586 [==============================] - 3s 5ms/step - loss: 0.0140 - val_loss: 0.0129 - lr: 0.0010
Epoch 4/100
586/586 [==============================] - 3s 5ms/step - loss: 0.0139 - val_loss: 0.0133 - lr: 0.0010
Epoch 5/100
586/586 [==============================] - 3s 5ms/step - loss: 0.0138 - val_loss: 0.0149 - lr: 0.0010
Epoch 6/100
586/586 [==============================] - 3s 5ms/step - loss: 0.0137 - val_loss: 0.0155 - lr: 0.0010
Epoch 7/100
586/586 [==============================] - 3s 5ms/step - loss: 0.0136 - val_loss: 0.0131 - lr: 0.0010
Epoch 8/100
586/586 [==============================] - 3s 5ms/step - loss: 0.0134 - val_loss: 0.0142 - lr: 0.0010
Epoch 9/100
586/586 [==============================] - 3s 5ms/step - loss: 0.0134 - val_


KeyboardInterrupt



In [10]:
plot_loss(history)

In [49]:
prediction = model.predict(np.array(x_train))
print("prediction shape:", prediction.shape)

2344/2344 [==============================] - 4s 2ms/step
prediction shape: (75000, 130)


In [66]:
i = np.random.randint(0, len(prediction))
pore_widths = np.load("data/initial kernels/Size_Kernel_Carbon_Adsorption.npy")
plt.plot(pore_widths, prediction[i], marker=".", label="Prediction")
plt.plot(pore_widths, y_train[i], marker=".", label="Real")
plt.legend()
plt.show()

In [41]:
import generator
gen = generator.Generator(path_s="data/initial kernels/Kernel_Carbon_Adsorption.npy",
                        path_d="data/initial kernels/Kernel_Carbon_Desorption.npy",
                        path_p_d="data/initial kernels/Pressure_Carbon.npy",
                        path_p_s="data/initial kernels/Pressure_Carbon.npy",
                        path_a="data/initial kernels/Size_Kernel_Carbon_Adsorption.npy"
                )
gen.pore_distribution = prediction[i]
gen.calculate_calculate_isotherms_right()
plt.plot(gen.pressures_s[min_exp_pressure_i:max_exp_pressure_i], x_train[i], marker=".", label="Real")
plt.plot(gen.pressures_s[min_exp_pressure_i:max_exp_pressure_i], gen.n_s[min_exp_pressure_i:max_exp_pressure_i], marker=".", label="Net")
plt.legend()
plt.show()

In [14]:
model.save('data/models/carbon2.keras')